# Medicare Claims Data Analysis Project Goals:

* Connect to an external/3rd party API and read data into your app

* Create and call at least 3 functions, at least one of which must return a value that is used **Possibly

* Calculate and display data based on an external factor (ex: get the current date, and display how many days remaining until some event) **Possibly

* Analyze text/data and display information about it (ex: how many words in a paragraph)

* Visualize data in a graph, chart, or other visual representation of data


In [14]:
import requests
import pandas as pd
import matplotlib as mpl


## Connect to an external/3rd party API and read data into your app

### Medicare Data Set of Surgical HCPCS Codes Reported on Claims, 2015-2017

* CMS API Endpoint, 2015: https://data.cms.gov/resource/uqfq-w9cg.json
* CMS API Endpoint, 2016: https://data.cms.gov/resource/jtra-d83c.json
* CMS API Endpoint, 2017: https://data.cms.gov/resource/2zuc-y5mm.json
* API Documentation, 2015: https://dev.socrata.com/foundry/data.cms.gov/uqfq-w9cg
* API Documentation, 2016: https://dev.socrata.com/foundry/data.cms.gov/jtra-d83c
* API Documentation, 2017: https://dev.socrata.com/foundry/data.cms.gov/2zuc-y5mm
* Was able to use documentation to format the below SoQL query within my link to get a specific set of HPCPCS Codes (10004 - 69990, surgical codes):
"https://data.cms.gov/resource/2zuc-y5mm.json?$where=hcpcs_code between '10004' and '69990'&$limit=10000 "

In [72]:
# Function that will retrieve raw data that can be read into dataframe.
def get_api_data(url):
    request = requests.get(url)
    return request.content


# Function that will format columns that need to be numeric so math operations can be run on them.
def format_df_types (df):
    df[["number_of_providers", 
        "number_of_services",
        "number_of_unique_beneficiary_provider_interactions",
        "number_of_distinct_medicare_beneficiary_per_day_services",
        "average_submitted_charge_amount",
        "average_medicare_allowed_amount",
        "average_medicare_payment_amount",
        "average_medicare_standardized_payment_amount"
       ]] = df[["number_of_providers", 
                "number_of_services",
                "number_of_unique_beneficiary_provider_interactions",
                "number_of_distinct_medicare_beneficiary_per_day_services",
                "average_submitted_charge_amount",
                "average_medicare_allowed_amount",
                "average_medicare_payment_amount",
                "average_medicare_standardized_payment_amount"
               ]].apply(pd.to_numeric)
    return df


# Function that will add columns needed for each dataframe that can be used for further analysis.
def add_totals_columns (df, year):
    df["total_billed_amount"] = df["number_of_services"] * df["average_medicare_allowed_amount"]
    df["total_amount_paid"] = df["number_of_services"] * df["average_medicare_payment_amount"]
    df["total_cost_to_patient"] = df["number_of_services"] * (df["average_medicare_allowed_amount"] - df["average_medicare_payment_amount"])
    df["year"] = year
    return df


df_2015 = format_df_types(pd.read_json(get_api_data("https://data.cms.gov/resource/uqfq-w9cg.json?$where=hcpcs_code between '10004' and '69990'&$limit=10000 ")))
df_2016 = format_df_types(pd.read_json(get_api_data("https://data.cms.gov/resource/jtra-d83c.json?$where=hcpcs_code between '10004' and '69990'&$limit=10000 ")))
df_2017 = format_df_types(pd.read_json(get_api_data("https://data.cms.gov/resource/2zuc-y5mm.json?$where=hcpcs_code between '10004' and '69990'&$limit=10000 ")))


df_2015 = add_totals_columns(df_2015, 2015)
df_2016 = add_totals_columns(df_2016, 2016)
df_2017 = add_totals_columns(df_2017, 2017)

frames = [df_2015, df_2016, df_2017]
df = pd.concat(frames)

# End result is a single master dataframe formatted with all columns we need, while original dataframes are available if needed.

In [73]:
df

,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,place_of_service,number_of_providers,number_of_services,number_of_unique_beneficiary_provider_interactions,number_of_distinct_medicare_beneficiary_per_day_services,average_submitted_charge_amount,average_medicare_allowed_amount,average_medicare_payment_amount,average_medicare_standardized_payment_amount,total_billed_amount,total_amount_paid,total_cost_to_patient,year
0,10021,Fine needle aspiration,N,F,2928,8256,7650,7941,365.07,72.13,55.24,54.90,595505.28,456061.44,139443.84,2015
1,10021,Fine needle aspiration,N,O,5673,17380,15357,16580,301.84,150.13,114.38,115.90,2609259.40,1987924.40,621335.00,2015
2,10022,Fine needle aspiration using imaging guidance,N,F,11160,93508,78994,80272,294.69,68.13,51.90,52.16,6370700.04,4853065.20,1517634.84,2015
3,10022,Fine needle aspiration using imaging guidance,N,O,6227,99842,66663,82098,312.60,145.89,111.81,110.10,14565949.38,11163334.02,3402615.36,2015
4,10030,Fluid collection drainage by catheter using im...,N,F,3514,7242,6677,6967,799.23,153.03,118.13,119.14,1108243.26,855497.46,252745.80,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7567,69960,Release ear canal,N,F,21,25,25,25,8188.68,1355.00,1059.10,962.43,33875.00,26477.50,7397.50,2017
7568,69970,Removal of growth of temporal bone,N,F,21,26,25,26,7649.44,1831.32,1430.14,1372.77,47614.32,37183.64,10430.68,2017
7569,69979,Temporal bone procedure,N,F,17,22,22,22,6777.40,1332.16,1038.33,1044.54,29307.52,22843.26,6464.26,2017
7570,69990,Use of operating microscope,N,F,3233,13709,13451,13700,1020.89,200.17,154.95,158.19,2744130.53,2124209.55,619920.98,2017


## Analyze text/data and display information about it (ex: how many words in a paragraph)

### What questions can I use this data set to answer?

* What surgical procedures were most common in 2017? (Can add more years if needed)
* What surgical procedures, overall, cost Medicare the most?
* What procedures were least covered (i.e. where discrepancy between charges and amount paid was greatest).
* What types of procedures are most common (can organize by specialty or by surgery method, etc.)
* What health problems does this analysis bring up and how can we solve them?

## Visualize data in a graph, chart, or other visual representation of data

* Will need to import library (such as matplotlib) for visualization) and find ways to use that.

## Other Goals

* Create and call at least 3 functions, at least one of which must return a value that is used **Possibly

* Calculate and display data based on an external factor (ex: get the current date, and display how many days remaining until some event) **Possibly

Will evaluate these to see if they would be practical in this project.